# 🚀 Text Generation Apps using Python (Completions Only)

This notebook showcases **14 text generation use cases** using OpenAI **Completions API**.  
Each section includes a description, the Python code, and expected output.

---

## 📑 Table of Contents
1. [Setup](#setup)
2. [Summarization](#uc1)  
3. [Sentiment Classification](#uc2)  
4. [Multilingual Generation + Translation](#uc3)  
5. [Semantic Interpretation of Idiom](#uc4)  
6. [Factual Recall / Explanatory Response](#uc5)  
7. [Code Generation & Explanation](#uc6)  
8. [Conversational Agent (FAQ Assistant)](#uc7)  
9. [Style Transfer / Tone Adaptation](#uc8)  
10. [Data-to-Text Generation](#uc9)  
11. [Creative Writing](#uc10)  
12. [Question Generation](#uc11)  
13. [Entity Extraction with Explanation](#uc12)  
14. [Paraphrasing / Rewriting](#uc13)  
15. [Email / Document Drafting](#uc14)  

---


<a id="setup"></a>
## Setup

- Ensure your `.env` file contains `OPENAI_API_KEY` (never hardcode secrets).
- Use `python-dotenv` to load environment variables securely.
- Import and initialize the `OpenAI` client with your API key.
- Define a reusable helper function `get_completion(prompt, ...)` to call the OpenAI Completions API with configurable model, temperature, and max tokens.
- Prefer environment variables for model selection and temperature overrides.
- Example setup and usage are shown in the next code cell.

In [ ]:
# Install required packages (skip if already installed)
# %pip install openai tiktoken python-dotenv

import os
import time
import random
import logging
from dataclasses import dataclass
from typing import Optional, Sequence

from dotenv import load_dotenv
from openai import OpenAI, APIError, RateLimitError

# -------------------------------------------------------------------
# Logging (avoid reconfiguring if handlers already exist)
# -------------------------------------------------------------------
if not logging.getLogger().handlers:
    logging.basicConfig(level=logging.INFO,
                        format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# -------------------------------------------------------------------
# Environment loading & helpers
# -------------------------------------------------------------------
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in environment or .env file")

DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-3.5-turbo-instruct")


def _env_float(name: str, default: float) -> float:
    raw = os.getenv(name)
    if not raw:
        return default
    # Ignore Windows TEMP style paths accidentally picked up
    if os.path.sep in raw:
        return default
    try:
        val = float(raw)
        if 0 <= val <= 1:
            return val
    except ValueError:
        pass
    return default


def _env_int(name: str, default: int, min_val: int = 1, max_val: int = 4000) -> int:
    raw = os.getenv(name)
    if not raw:
        return default
    try:
        val = int(raw)
        if min_val <= val <= max_val:
            return val
    except ValueError:
        pass
    return default


def _get_client() -> OpenAI:
    # Could be extended for per-request customization later
    return OpenAI(api_key=OPENAI_API_KEY)


client = _get_client()

# -------------------------------------------------------------------
# Result container
# -------------------------------------------------------------------


@dataclass
class CompletionResult:
    text: str
    model: str
    prompt_tokens: Optional[int] = None
    completion_tokens: Optional[int] = None
    total_tokens: Optional[int] = None
    retries: int = 0

# -------------------------------------------------------------------
# Core helper with light retries
# -------------------------------------------------------------------


def get_completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    max_tokens: Optional[int] = None,
    temperature: Optional[float] = None,
    stop: Optional[Sequence[str]] = None,
    log_response: bool = False,
    max_retries: int = 3,
    backoff_base: float = 1.5,
) -> Optional[CompletionResult]:
    """Call OpenAI Completions API with resilience & env overrides.

    Environment overrides:
      OPENAI_MODEL          -> default model if not passed
      OAI_TEMP / OAI_TEMPERATURE -> temperature (0..1)
      OAI_MAX_TOKENS        -> max_tokens upper bound
    """
    if not prompt or not prompt.strip():
        raise ValueError("Prompt must be a non-empty string")

    # Apply env fallbacks only if caller did not specify
    if temperature is None:
        temperature = _env_float(
            "OAI_TEMP", _env_float("OAI_TEMPERATURE", 0.7))
    if max_tokens is None:
        max_tokens = _env_int("OAI_MAX_TOKENS", 200, 1, 4096)

    attempt = 0
    last_error: Optional[Exception] = None
    while attempt <= max_retries:
        try:
            start = time.time()
            resp = client.completions.create(
                model=model,
                prompt=prompt.strip(),
                max_tokens=max_tokens,
                temperature=temperature,
                stop=list(stop) if stop else None,
            )
            latency = (time.time() - start) * 1000
            choice = resp.choices[0]
            usage = getattr(resp, "usage", None)
            if log_response:
                logger.debug("Raw response: %s", resp)
            logger.info(
                "completion model=%s tokens=%s latency=%.1fms retries=%d",  # noqa: E501
                model,
                getattr(usage, "total_tokens", "?"),
                latency,
                attempt,
            )
            return CompletionResult(
                text=choice.text.strip(),
                model=model,
                prompt_tokens=getattr(usage, "prompt_tokens", None),
                completion_tokens=getattr(usage, "completion_tokens", None),
                total_tokens=getattr(usage, "total_tokens", None),
                retries=attempt,
            )
        except (RateLimitError, APIError) as e:
            last_error = e
            if attempt == max_retries:
                logger.error("Max retries reached (%d): %s", max_retries, e)
                break
            sleep_for = backoff_base ** attempt + random.uniform(0, 0.25)
            logger.warning("Transient error (%s). Retrying in %.2fs (attempt %d/%d)",
                           e.__class__.__name__, sleep_for, attempt + 1, max_retries)
            time.sleep(sleep_for)
            attempt += 1
            continue
        except Exception as e:  # Non-retryable
            logger.error("Completion failed: %s", e)
            last_error = e
            break
    return None


# -------------------------------------------------------------------
# Example usage (kept minimal for notebook demonstration)
# -------------------------------------------------------------------
if __name__ == "__main__":
    test_prompt = "Write a haiku about AI and the future."
    result = get_completion(test_prompt, log_response=True)
    if result:
        print("✅ Completion result:\n", result.text)
        if result.total_tokens is not None:
            print(
                f"(tokens: {result.total_tokens}, retries: {result.retries})")
    else:
        print("❌ No response received.")

<a id="uc1"></a>
## ✅ Use Case 1: Summarization
**Input:**  
Sparse vector feature indexing allows scalable search over events by turning categorical attributes into an extremely wide, mostly empty vector...

**Sample Output:**  
"It converts categorical features into sparse vectors for scalable search. The trade-off is efficiency vs. memory/compute cost at high cardinality."

In [ ]:
# ✅ Use Case 1: Summarization

technical_paragraph = """
Sparse vector feature indexing allows scalable search over events by turning 
categorical attributes into an extremely wide, mostly empty vector. Each distinct 
token (e.g. event_type:click, browser:mobile) maps to a coordinate set to 1 while 
all others remain 0. This removes unintended ordering between categories and keeps 
distance metrics meaningful, but at very high cardinality memory and compute 
overhead grow rapidly, motivating hashing tricks or learned embeddings.
""".strip()

summary_prompt = f"Summarize the following paragraph in 2 crisp sentences focusing on purpose and trade-offs:\n\n{technical_paragraph}\n\nSummary:"

# Call completions API using helper
summary = get_completion(
    summary_prompt, max_tokens=120, temperature=0.4)

print("✅ Summary:", summary)

<a id="uc2"></a>
## ✅ Use Case 2: Sentiment Classification
**Input:**  
"The user interface feels sluggish, but the reporting features are fantastic."

**Sample Output:**  
"Neutral"


In [ ]:
# ✅ Use Case 2: Sentiment Classification (Multiple Inputs)

# Sample inputs
input_texts = [
    "The user interface feels sluggish, but the reporting features are fantastic.",  # Neutral
    "I love the new dashboard design, it’s clean and very intuitive.",               # Positive
    "The system keeps crashing frequently, making it hard to use."                   # Negative
]

# Loop through each input and classify sentiment
for input_text in input_texts:
    sentiment_prompt = f"""
    Classify the sentiment of the following text strictly as Positive, Negative, or Neutral.
    If the text contains both positive and negative aspects, classify it as Neutral.

    Text: "{input_text}"

    Sentiment:
    """

    # Call completions API using helper (returns CompletionResult or None)
    result = get_completion(
        sentiment_prompt,
        max_tokens=20,
        temperature=0  # deterministic output
    )

    if result and result.text:
        sentiment_text = result.text.strip()
    else:
        sentiment_text = "(no response)"

    print(f"✅ Text: {input_text}")
    print(f"✅ Sentiment: {sentiment_text}\n")

<a id="uc3"></a>
## ✅ Use Case 3: Multilingual Generation + Translation
**Input:**  
"Artificial Intelligence will transform education."

**Sample Output:**  
- French: "L'intelligence artificielle transformera l'éducation."  
- Spanish: "La inteligencia artificial transformará la educación."  
- Hindi: "कृत्रिम बुद्धिमत्ता शिक्षा को बदल देगी।"


In [ ]:
# ✅ Use Case 3: Multilingual Generation + Translation

input_text = "Artificial Intelligence will transform education."
languages = ["French", "Spanish", "Hindi"]

for lang in languages:
    translation_prompt = f"Translate the following text into {lang}:\n\n{input_text}"
    translation = get_completion(
        translation_prompt, max_tokens=60, temperature=0.3)
    print(f"✅ {lang}: {translation}\n\n")

<a id="uc4"></a>
## ✅ Use Case 4: Semantic Interpretation of Idiom
**Input:**  
"Kick the bucket"

**Sample Output:**  
"It means someone has died."


In [ ]:
# ✅ Use Case 4: Semantic Interpretation of Idiom

idioms = [
    "kick the bucket",
    "spill the beans",
    "hit the sack",
    "once in a blue moon",
    "break the ice",
]

for idiom in idioms:
    prompt = f"""
Explain the meaning of the idiom below in one concise, literal-friendly plain English sentence.
Avoid extra commentary or cultural notes unless essential.
Return ONLY the meaning (no leading labels).

Idiom: "{idiom}"

Meaning:
""".strip()

    meaning = get_completion(
        prompt,
        model="gpt-3.5-turbo-instruct",
        max_tokens=60,
        temperature=0  # deterministic for stable definitions
    )

    if meaning:
        print(f"🗣️ {idiom} -> {meaning}\n\n")
    else:
        print(f"❌ No response for idiom: {idiom}\n\n")

<a id="uc5"></a>
## ✅ Use Case 5: Factual Recall / Explanatory Response
**Input:**  
"Why is the sky blue?"

**Sample Output:**  
"The sky looks blue because sunlight is scattered by air molecules. Blue light is scattered more strongly than other colors, so we mostly see blue."

In [ ]:
# ✅ Use Case 5: Factual Recall / Explanatory Response

question = "Why is the sky blue?"

prompt = f"""
Provide a concise scientific explanation (2 sentences) for the question below.
Avoid fluff; focus on the underlying physical phenomenon.

Question: {question}

Answer:
""".strip()

result = get_completion(prompt, max_tokens=120, temperature=0.2)

if result and result.text:
    print("❓ Question:", question)
    print("🧪 Answer:", result.text.strip())
    if result.total_tokens is not None:
        print(f"(tokens: {result.total_tokens}, retries: {result.retries})")
else:
    print("❌ No response received.")

<a id="uc6"></a>
## ✅ Use Case 6: Code Generation & Explanation
**Input:**  
"Write a Python function that checks if a number is prime."

**Sample Output:**  
```python
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
````

**Explanation:**
"The function checks divisibility from 2 up to √n. If any divisor is found, it returns False; otherwise, True."


In [ ]:
# ✅ Use Case 6: Code Generation & Explanation

problem = "Write a Python function that checks if a number is prime."

prompt = f"""
You are a senior Python engineer.
Generate clean, production-quality code that fulfills the requirement below.
After the code, add a single concise explanation sentence.
Use exactly this delimiter line before the explanation:
---EXPLANATION---

Requirement:
{problem}

Return ONLY:
1. A single Python code block
2. The delimiter line
3. One concise explanation sentence
""".strip()

result = get_completion(prompt, max_tokens=220, temperature=0)

if not result or not result.text:
    print("❌ No response received.")
else:
    raw = result.text.strip()
    # Attempt to split code and explanation
    parts = raw.split("---EXPLANATION---", 1)
    code_part = raw
    explanation = None
    if len(parts) == 2:
        code_part, explanation = parts[0].strip(), parts[1].strip()

    print("🧩 Generated Code:")
    print(code_part)
    if explanation:
        print("\n📝 Explanation:", explanation)
    if result.total_tokens is not None:
        print(f"\n(tokens: {result.total_tokens}, retries: {result.retries})")

<a id="uc7"></a>
## ✅ Use Case 7: Conversational Agent (FAQ Assistant)

**Input:**
"What is the purpose of Git?"

**Sample Output:**
"Git is a version control system that tracks changes in code, helps collaboration, and manages project history."

In [ ]:
# ✅ Use Case 7: Conversational Agent (FAQ Assistant)

# A tiny in-memory FAQ knowledge base (could be loaded from file/db in real apps)
faq_kb = {
    "What is Git?": "Git is a distributed version control system for tracking changes in source code.",
    "Why use version control?": "It enables history, collaboration, branching, and safe experimentation.",
    "What is a commit?": "A commit records a snapshot of your staged changes with a message.",
    "What is branching?": "Branching lets you diverge from the main line of development to work independently.",
    "What is a pull request?": "A pull request proposes merging changes from one branch into another, supporting review.",
}

# Simulated user questions (some exact, some paraphrased / partially overlapping)
user_queries = [
    "What is the purpose of Git?",
    "Explain branching in simple terms.",
    "Why should teams adopt version control?",
]

system_instruction = (
    "You are a concise technical FAQ assistant. Answer ONLY from the provided FAQ entries. "
    "If the answer is not present, reply with 'I don't have that information.' Keep answers to one or two sentences."
)

history: list[tuple[str, str]] = []  # (user_question, answer)

for q in user_queries:
    # Build a retrieval-aware prompt by listing KB and prior turns.
    kb_block = "\n".join(f"Q: {k}\nA: {v}" for k, v in faq_kb.items())
    convo_block = "\n".join(
        # last few turns
        f"User: {hq}\nAssistant: {ha}" for hq, ha in history[-4:])

    prompt = f"""
{system_instruction}

FAQ Knowledge Base:
{kb_block}

Conversation So Far:
{convo_block if convo_block else '(no previous turns)'}

User Question: {q}

Answer (draw ONLY from KB or say you don't have it):
""".strip()

    result = get_completion(prompt, max_tokens=110, temperature=0.2)

    if result and result.text:
        answer = result.text.strip()
    else:
        answer = "(no response)"

    history.append((q, answer))
    print(f"🙋 User: {q}")
    print(f"🤖 Assistant: {answer}\n")

# Show a compact transcript summary
print("--- Transcript Summary ---")
for uq, ans in history:
    print(f"Q: {uq}\nA: {ans}\n")


## ✅ Use Case 8: Style Transfer / Tone Adaptation

**Input:**
"Please confirm receipt of this document at the earliest."

**Sample Output:**
"Hey, just let me know when you get this doc!"

In [ ]:
# ✅ Use Case 8: Style Transfer / Tone Adaptation

original_text = "Please confirm receipt of this document at the earliest."

# Define target tones / style adaptation goals
style_specs = [
    {
        "label": "Casual Friendly",
        "instruction": "Rewrite to a casual, friendly tone as if messaging a colleague you know well. Conserve meaning."
    },
    {
        "label": "Professional Polite",
        "instruction": "Rewrite in a concise, professional, polite corporate email tone. Avoid urgency exaggeration."
    },
    {
        "label": "Enthusiastic Marketing",
        "instruction": "Rewrite with upbeat, energetic marketing-style enthusiasm while keeping the core request intact."
    },
    {
        "label": "Concise SMS",
        "instruction": "Rewrite as a very short SMS text (≤ 14 words), plain language, no emojis."
    },
]

# Temperature per style (defaults if not specified)
style_temperature = {
    "Casual Friendly": 0.6,
    "Professional Polite": 0.2,
    "Enthusiastic Marketing": 0.8,
    "Concise SMS": 0.3,
}

print(f"📄 Original: {original_text}\n")

for spec in style_specs:
    label = spec["label"]
    instruction = spec["instruction"]
    temperature = style_temperature.get(label, 0.5)

    prompt = f"""
You are an expert tone adaptation assistant.
Task: {instruction}

Constraints:
- Preserve the original factual meaning.
- Do NOT add new details, placeholders, or emojis unless specified.
- Output ONLY the rewritten sentence (no quotes, no label, no explanation).

Original: {original_text}

Rewritten:
""".strip()

    result = get_completion(prompt, max_tokens=60, temperature=temperature)

    if result and result.text:
        print(f"🎯 {label}: {result.text.strip()}\n\n")
    else:
        print(f"❌ {label}: (no response)\n\n")

# (Optional) You could adapt this into a function or batch process for many sentences.

## ✅ Use Case 9: Data-to-Text Generation

**Input:**
Region: North
Sales: \$50,000
Increase: 10%

**Sample Output:**
"The North region generated \$50,000 in sales, reflecting a 10% increase compared to the previous period."


In [ ]:
# ✅ Use Case 9: Data-to-Text Generation
"""Generate natural language summaries from structured tabular data.
This demonstrates:
- Turning rows into fluent sentences
- Enforcing style & constraints
- Producing both per-row narratives and an aggregate insight
"""

# Example structured data (could come from a dataframe / CSV)
# Amounts in USD
regional_metrics = [
    {"region": "North", "sales": 50000, "growth_pct": 10.0, "new_customers": 42},
    {"region": "South", "sales": 38000, "growth_pct": 4.5, "new_customers": 35},
    {"region": "West",  "sales": 61000, "growth_pct": 12.3, "new_customers": 57},
    {"region": "East",  "sales": 45500, "growth_pct": 7.1, "new_customers": 48},
]

# Helper to format data block for the prompt (kept deterministic)


def format_data(rows: list[dict]) -> str:
    lines = ["Region | Sales | GrowthPct | NewCustomers"]
    for r in rows:
        lines.append(
            f"{r['region']} | {r['sales']} | {r['growth_pct']} | {r['new_customers']}")
    return "\n".join(lines)


data_block = format_data(regional_metrics)

per_row_instruction = (
    "For each region produce ONE sentence: 'The <Region> region generated $X in sales (Y% growth) adding Z new customers.' "
    "Use commas sparingly. No extra commentary."
)

aggregate_instruction = (
    "After listing sentences, add a final concise aggregate insight comparing best vs weakest growth and overall momentum. "
    "Do NOT restate raw table values verbatim beyond what is needed."
)

prompt = f"""
You are a data summarization assistant.
Transform the structured table into natural language.

Table (pipe-delimited):
{data_block}

Instructions:
- {per_row_instruction}
- {aggregate_instruction}
- Keep total output under 8 sentences.
- Tone: objective, analyst brief.
- Output only the sentences (no bullets, no headings).

Output:
""".strip()

result = get_completion(prompt, max_tokens=260, temperature=0.25)

if not result or not result.text:
    print("❌ No response received.")
else:
    print("📝 Data-to-Text Summary:\n")
    print(result.text.strip())
    if result.total_tokens is not None:
        print(f"\n(tokens: {result.total_tokens}, retries: {result.retries})")

# Optional: show how you might adapt a single-row quick narrative


def single_row_narrative(row: dict) -> str:
    p = (
        "Rewrite the following structured record as one concise sentence summarizing performance. "
        "Use present tense. No embellishment. Return only the sentence."  # deterministic pattern
    )
    record_block = ", ".join(f"{k}={v}" for k, v in row.items())
    single_prompt = f"{p}\nRecord: {record_block}\nSentence:"
    single_res = get_completion(single_prompt, max_tokens=50, temperature=0.2)
    return single_res.text.strip() if single_res and single_res.text else "(no result)"


print("\n--- Single Row Example (West) ---")
print(single_row_narrative(regional_metrics[2]))

## ✅ Use Case 10: Creative Writing

**Input:**
"A robot learning to paint"

**Sample Output:**
"The robot dipped its brush in bright colors, each stroke a clumsy attempt at beauty. Slowly, it learned to create sunsets and flowers. Its creators realized the machine had found its soul in art."

In [ ]:
# ✅ Use Case 10: Creative Writing
"""Generate multiple stylistic story variants from the same seed concept.
Demonstrates:
- Controlling style & tone via instructions
- Adjusting creativity using temperature
- Lightweight post-processing (title suggestion)
"""

story_seed = "A robot learning to paint"

style_specs = [
    {
        "label": "Concise Vignette",
        "instruction": "Write 3 crisp sentences, understated and cinematic. Subtle emotion, no dialogue.",
        "temperature": 0.65,
        "max_tokens": 160,
    },
    {
        "label": "Descriptive Sci‑Fi",
        "instruction": "Write 4 sentences with tactile sensory detail and soft speculative imagery. Mild wonder, no techno-babble.",
        "temperature": 0.85,
        "max_tokens": 190,
    },
    {
        "label": "Whimsical Children's Book",
        "instruction": "Write 3–4 short, playful sentences a child can understand. Gentle personification, warm ending.",
        "temperature": 0.9,
        "max_tokens": 170,
    },
    {
        "label": "Reflective Literary",
        "instruction": "Write one compact paragraph (4 sentences) with lyrical cadence and a quiet epiphany in the final line.",
        "temperature": 0.8,
        "max_tokens": 210,
    },
]

print(f"🎨 Seed Concept: {story_seed}\n")
all_variants: list[tuple[str, str]] = []  # (label, text)

for spec in style_specs:
    label = spec["label"]
    instruction = spec["instruction"]
    temp = spec["temperature"]
    max_toks = spec["max_tokens"]

    prompt = f"""
You are an accomplished creative writer.
Task: {instruction}
Seed Concept: "{story_seed}"
Constraints:
- Avoid clichés (e.g., 'outside the box', 'glitch in the system').
- Keep the passage self-contained (no need for title).
- No explicit moral statements.
- Return ONLY the passage (no label, no meta commentary).

Passage:
""".strip()

    result = get_completion(prompt, max_tokens=max_toks, temperature=temp)

    if not result or not result.text:
        print(f"❌ {label}: (no response)\n")
        continue

    text = result.text.strip()
    all_variants.append((label, text))
    print(f"🖋️ {label} (temp={temp}):\n{text}\n")

# Optional: Generate a concise evocative title for the strongest variant (choose the Reflective Literary if present)
reflective = next(
    (t for (lbl, t) in all_variants if lbl.startswith("Reflective")), None)
if reflective:
    title_prompt = f"Provide a 5-word evocative literary title (Title Case) for the following passage. No quotes.\n\nPassage:\n{reflective}\n\nTitle:".strip(
    )
    title_res = get_completion(title_prompt, max_tokens=12, temperature=0.6)
    if title_res and title_res.text:
        print("🏷️ Suggested Title:", title_res.text.strip())

## ✅ Use Case 11: Question Generation

**Input:**
"The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south."

**Sample Output:**

1. What is the largest ocean on Earth?
2. How deep is the Pacific Ocean compared to other oceans?
3. Which oceans border the Pacific to the north and south?


In [ ]:
# ✅ Use Case 11: Question Generation
"""Generate study / assessment questions from a source passage.
Demonstrates:
- Mapping a single text to multiple cognitive levels (factual, inferential, analytical)
- Controlling format and difficulty tiers
- Light post-processing to ensure clean numbering
"""

source_passage = (
    "The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. "
    "It stretches from the Arctic in the north to the Southern Ocean near Antarctica in the south, "
    "and is bounded by Asia and Australia in the west and the Americas in the east. Its vast size "
    "influences global climate patterns, marine biodiversity, and trade routes."
)

# Specification for desired questions
question_plan = [
    ("Factual Recall", 2, "Direct facts explicitly stated (who / what / where)."),
    ("Conceptual / Inferential", 2,
     "Questions requiring connecting two statements or reasoning implicit relationships."),
    ("Analytical / Higher-Order", 1,
     "Question prompting implications, significance, or broader impact."),
]


def build_instruction(plan):
    lines = ["Generate high-quality questions from the passage.", "Rules:"]
    lines += [
        "- No answers in output.",
        "- Each question must be self-contained (can stand alone).",
        "- Avoid redundancy and avoid yes/no questions.",
        "- Use clear academic English.",
        "- Do NOT mention the tiers in the output; just list questions numbered.",
    ]
    lines.append("Desired distribution (internal guide, not to print):")
    for tier, count, desc in plan:
        lines.append(f"  * {tier}: {count} – {desc}")
    return "\n".join(lines)


instruction_block = build_instruction(question_plan)

prompt = f"""
You are an educational content designer.
Passage:\n{source_passage}\n\n{instruction_block}\n\nOutput ONLY numbered questions (1., 2., ...). Start numbering at 1.
Questions:
""".strip()

result = get_completion(prompt, max_tokens=220, temperature=0.3)

if not result or not result.text:
    print("❌ No response received.")
else:
    raw = result.text.strip()
    # Post-process: ensure numbering starts at 1 and is sequential
    lines = [l.strip() for l in raw.splitlines() if l.strip()]
    cleaned = []
    counter = 1
    for ln in lines:
        # Remove existing leading number patterns like '1.' or '1)' or '(1)'
        stripped = ln
        # Simple patterns
        if stripped[0:3].isdigit():
            pass  # rare case; handle generically below
        import re
        stripped = re.sub(r"^\(?\d+[\).]\s*", "", stripped)
        cleaned.append(f"{counter}. {stripped}")
        counter += 1
    print("📝 Generated Questions:\n")
    print("\n".join(cleaned))
    if result.total_tokens is not None:
        print(f"\n(tokens: {result.total_tokens}, retries: {result.retries})")

# (Optional) A quick function to generate n factual-only questions (could be reused elsewhere)


def generate_factual_only(passage: str, n: int = 3) -> list[str]:
    p = f"Generate {n} distinct factual recall questions (no answers) from the passage. Number them. Passage: {passage}\nQuestions:"  # noqa: E501
    r = get_completion(p, max_tokens=120, temperature=0.25)
    if not r or not r.text:
        return []
    return [ln.split('.', 1)[1].strip() if '.' in ln else ln.strip() for ln in r.text.splitlines() if ln.strip()]


# Example usage of helper (not required for main output)
extra = generate_factual_only(source_passage, 2)
if extra:
    print("\n🔎 Extra Factual Questions:")
    for q in extra:
        print(f"- {q}")

## ✅ Use Case 12: Entity Extraction with Explanation

**Input:**
"Apple announced a new partnership with Tesla to develop AI-powered batteries in California."

**Sample Output:**

* **Apple** – Technology company forming a partnership.
* **Tesla** – Automotive/energy company collaborating on batteries.
* **California** – Location where the initiative is happening.

In [ ]:
# ✅ Use Case 12: Entity Extraction with Explanation
"""Extract named entities with short functional explanations.
Demonstrates:
- Instruction for schema-like output
- Deterministic extraction (low temperature)
- Light post-processing to normalize formatting
"""

text = (
    "Apple announced a new partnership with Tesla to develop AI-powered batteries in California. "
    "The collaboration will focus on scalable sustainable energy storage, leveraging Apple's design "
    "ecosystem and Tesla's manufacturing expertise."
)

prompt = f"""
You are an information extraction assistant.
Task: Identify distinct real-world entities in the passage and list each with:
- Canonical surface form
- Type (choose from: Organization, Location, Technology, Initiative)
- One concise explanation (≤ 15 words) describing its role in context.

Rules:
- Do not invent entities not present.
- Do not repeat the same entity.
- If something is both a project and a company, choose the most salient type for this context.
- Avoid marketing language; be factual.

Return ONLY a Markdown bullet list in the form:
* Entity – Type: Explanation

Passage:
{text}

List:
""".strip()

result = get_completion(prompt, max_tokens=220, temperature=0.15)

if not result or not result.text:
    print("❌ No response received.")
else:
    raw = result.text.strip()
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    # Normalize: ensure bullets start with '* '
    normalized = []
    for ln in lines:
        if not ln.startswith(('*', '-')):
            ln = '* ' + ln.lstrip('*- ')  # force bullet
        else:
            # replace leading '-' with '*'
            ln = '* ' + ln.lstrip('*- ').strip()
        normalized.append(ln)
    print("🧾 Extracted Entities:\n")
    print("\n".join(normalized))
    if result.total_tokens is not None:
        print(f"\n(tokens: {result.total_tokens}, retries: {result.retries})")

# Optional helper for reuse


def extract_entities(passage: str) -> list[tuple[str, str, str]]:
    """Return list of (entity, type, explanation)."""
    p = f"Identify entities with type and ≤10-word explanation. Passage: {passage}\nList:"  # noqa: E501
    r = get_completion(p, max_tokens=140, temperature=0.2)
    if not r or not r.text:
        return []
    triples = []
    import re
    for line in r.text.splitlines():
        line = line.strip('*- ').strip()
        if not line:
            continue
        # Pattern: Entity – Type: Explanation
        m = re.match(r"(.+?)\s+[–-]\s+(\w+):\s+(.+)", line)
        if m:
            triples.append((m.group(1).strip(), m.group(
                2).strip(), m.group(3).strip()))
    return triples


# Quick demonstration (not required)
_demo = extract_entities(text)
if _demo:
    print("\n🔍 Parsed Triples:")
    for ent, typ, expl in _demo:
        print(f"- {ent} ({typ}): {expl}")

## ✅ Use Case 13: Paraphrasing / Rewriting

**Input:**
"The proliferation of interconnected devices has exponentially increased the complexity of ensuring cybersecurity across distributed networks."

**Sample Output:**
"With more connected devices, keeping networks secure has become much harder."


In [ ]:
# ✅ Use Case 13: Paraphrasing / Rewriting
"""Generate multiple paraphrase variants with different constraints.
Demonstrates:
- Style-controlled rewriting
- Brevity vs. fidelity trade-offs
- Enforcing no meaning drift
"""

original_sentence = (
    "The proliferation of interconnected devices has exponentially increased the complexity of ensuring "
    "cybersecurity across distributed networks."
)

rewrite_specs = [
    {
        "label": "Plain Concise",
        "instruction": "Rewrite in everyday language, ≤ 22 words, preserve meaning.",
        "temperature": 0.35,
    },
    {
        "label": "Executive Summary",
        "instruction": "Rewrite as a crisp executive summary (≤ 18 words) highlighting the core risk.",
        "temperature": 0.4,
    },
    {
        "label": "Technical Precision",
        "instruction": "Rewrite with precise technical wording, neutral tone, no adjectives, ≤ 28 words.",
        "temperature": 0.3,
    },
    {
        "label": "Minimal Compression",
        "instruction": "Compress to ≤ 12 words retaining key idea.",
        "temperature": 0.45,
    },
]

print("🧾 Original:\n" + original_sentence + "\n")

for spec in rewrite_specs:
    prompt = f"""
You are a rewriting assistant.
Task: {spec['instruction']}
Rules:
- No new facts.
- Do not remove the notion of increased complexity or distributed networks.
- Output ONLY the rewritten sentence (no label, no quotes).

Original: {original_sentence}

Rewritten:
""".strip()
    res = get_completion(prompt, max_tokens=70,
                         temperature=spec['temperature'])
    if res and res.text:
        print(f"✏️ {spec['label']}: {res.text.strip()}\n")
    else:
        print(f"❌ {spec['label']}: (no response)\n")

# Optional: Quick single paraphrase helper


def paraphrase(text: str, style: str = "neutral concise") -> str:
    p = f"Rewrite in a {style} style without changing meaning. Text: {text}\nRewritten:"  # noqa: E501
    r = get_completion(p, max_tokens=60, temperature=0.4)
    return r.text.strip() if r and r.text else "(no result)"


print("--- Helper Demo ---")
print(paraphrase(original_sentence, "succinct plain English"))


## ✅ Use Case 14: Email / Document Drafting

**Input:**
"Announce a team meeting for Friday at 3 PM to discuss the quarterly results."

**Sample Output:**
"Subject: Team Meeting – Friday at 3 PM
Hi Team,
We will have a meeting on Friday at 3 PM to review our quarterly results. Please make sure to attend.
Best regards,
\[Your Name]"



In [ ]:
# ✅ Use Case 14: Email / Document Drafting
"""Draft a professional email from a concise instruction.
Demonstrates:
- Structured multi-section output
- Tone control and placeholders
- Optional variant generation (formal vs. friendly)
"""

instruction = "Announce a team meeting for Friday at 3 PM to discuss the quarterly results."

prompt = f"""
You are an assistant that drafts clear professional internal emails.
Instruction: {instruction}

Requirements:
- Include a subject line (prefix with 'Subject:').
- Opening greeting addressing 'Team'.
- Body: purpose, meeting details (date, time, topic), expectation to attend, optional prep mention.
- Closing with a neutral professional sign-off (e.g., 'Best regards,').
- Insert placeholder '[Your Name]' for sender.
- Keep to ≤ 140 words.
- No exaggerated enthusiasm or emojis.

Return ONLY the email text. No markdown fences.

Email:
""".strip()

result = get_completion(prompt, max_tokens=260, temperature=0.35)

if not result or not result.text:
    print("❌ No response received.")
else:
    email = result.text.strip()
    print("📧 Draft Email:\n")
    print(email)
    if result.total_tokens is not None:
        print(f"\n(tokens: {result.total_tokens}, retries: {result.retries})")

# Optional: Generate a friendlier variant
variant_prompt = f"Rewrite the following email in a slightly friendlier but still professional tone. No added details.\nEmail:\n{email}\n\nRewritten:" if result and result.text else None
if variant_prompt:
    variant_res = get_completion(
        variant_prompt, max_tokens=240, temperature=0.55)
    if variant_res and variant_res.text:
        print("\n🤝 Friendly Variant:\n")
        print(variant_res.text.strip())